## Wikipediaコーパスを利用した Word2vec による関係要素のembedding

In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import seaborn as sns
from matplotlib import ticker
from matplotlib.ticker import ScalarFormatter

from gensim.models import Word2Vec

In [ ]:
# プロットする図のサイズ設定
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams["font.size"] = 13
# プロットする図のフォント設定
font_manager.fontManager.addfont('/Library/Fonts/ipaexg.ttf')
rc('font', family='IPAEXGothic')

sns.set_context('talk')
sns.set_style("ticks") # スタイルをticksに
sns.set(context='talk', style='ticks', font=["IPAEXGothic"], font_scale=10/6, )

- データの読み込み

In [ ]:
# data/result/に格納されている全csvファイルの読み込み
csv_files = glob.glob(os.path.join("./data/", "*.csv"))
df_list = []
for file in csv_files:
    tmp_df = pd.read_csv(file)
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)

df.shape

- 「property要素 + not (wikiPage) + not (画像)」 の関係リンクを持つデータの抽出

In [ ]:
# predの要素で, propertyが含まれる値を返す
df_prep = df[df.pred.str.contains('property')]

# 'wikiPage'を含まないprepertyを返す
df_prep = df_prep[~df_prep['pred'].str.contains('wikiPage')]

# '画像'を含まないprepertyを返す
df_prep = df_prep[~df_prep['pred'].str.contains('画像')]

df_prep.shape

- 前処理

In [ ]:
# Nan削除
df_prep = df_prep.dropna()

- objデータが数値を持つデータを省く

In [ ]:
# objデータに'数値'を含まない値を返す
df_prep = df_prep[df_prep['obj'].apply(lambda x: pd.to_numeric(x, errors='coerce')).isnull()]
        
df_prep.shape

- Word2vec のmodel 読み込み

In [ ]:
# gensim.model の読み込み
# 事前学習済みのモデルや提供されているモデルのパスを記入

model = Word2Vec.load('../Models/japanese-word2vec-model-builder/word2vec.gensim.model')

In [ ]:
# モデルを利用した文字のエンべディング
## 未知語に関しては, 今回はNaNで対応
def vectorize(model, word):
    try:
        output = model.wv[word]
        return output
    except:
        return np.nan

In [ ]:
# obj要素に関してはLOD同士が繋がっているため, 基本的にURIで記述されている.
# また, (県の魚:〇〇)の要素や数値データが入っている場合もあるため, それぞれ最後尾の要素を値として扱う. 
def preprocessing(obj):
    if type(obj) != str:
        output = obj
    else:
        output = obj.split("/")[-1]
    return output

- word2vec でのエンべディングを実施する
    - key, obj から pred(rel) の分散表現ベクトルを求める

In [ ]:
## key のvectorization
### key に関しては,preprocessingは特に必要ないと仮定

df_prep['key_vec'] = df_prep['key'].map(lambda key:vectorize(model, key))
is_key_notnull = df_prep['key_vec'].notnull()
print("登録済みkeyの要素数は:",len(df_prep[is_key_notnull]))
print("登録済みkeyのユニーク要素の個数は:", \
      df_prep[is_key_notnull]['key'].nunique())

In [ ]:
## obj のpreprocessing
  ## DBpedia上で登録されているuri情報を取り除く

uri = 'http://ja.dbpedia.org/resource/'
obj_list = df_prep['obj'].map(lambda obj:obj.replace(uri,''))
obj_list = obj_list.map(lambda obj:obj.replace('※',''))
obj_list = obj_list.str.replace('[()]','', regex=True)
obj_list = obj_list.str.replace('[（）]',' ', regex=True)
obj_list = obj_list.str.replace('.+[「『]|[」』]','', regex=True)
obj_list = obj_list.str.strip()

In [ ]:
def obj_preprocessing_1(obj):
    if type(obj) != str:
        output = obj
    elif len(obj.split("：")) != 1:
        output = obj.split("：")[1]
    elif len(obj.split(":")) != 1:
        output = obj.split(":")[1]
    else:
        output = obj
    return output.strip()  
        
def obj_preprocessing_2(obj):
    if type(obj) != str:
        output = obj
    elif len(obj.split("_")) != 1:
        output = obj.split("_")[0]
    elif len(obj.split("、")) != 1:
        output = obj.split("、")[0]
    elif len(obj.split(" ")) != 1:
        output = obj.split(" ")[0]
    elif len(obj.split("、")) != 1:
        output = obj.split("、")[0]
    elif len(obj.split("・")) != 1:
        output = obj.split("・")[0]
    elif len(obj.split("・")) != 1:
        output = obj.split("・")[0]
    else:
        output = obj
    return output.strip()

In [ ]:
obj_list = obj_list.map(lambda obj:obj_preprocessing_1(obj))
obj_list = obj_list.map(lambda obj:obj_preprocessing_2(obj))
df_prep['obj_vec'] = obj_list.map(lambda obj:vectorize(model, obj))
df_prep['pre_obj'] = obj_list
is_obj_notnull = df_prep['obj_vec'].notnull()
print("登録済みobjの要素数は:",len(df_prep[is_obj_notnull]))
print("登録済みobjのユニーク要素の個数は:", \
      df_prep[is_obj_notnull]['obj'].nunique())

- key + rel = obj が成り立つと仮定する.
  - そこで, rel は, obj - key で求められるとする.
  - その後, 求めたrel を要素ごとの値ではなく関係ごとの値として定める.

In [ ]:
df_prep = df_prep.loc[is_key_notnull & is_obj_notnull]
df_prep['pred_vec'] = df_prep['obj_vec'] - df_prep['key_vec']
print("pred_vec がnullの要素数は：",df_prep['pred_vec'].isnull().sum())

In [ ]:
df_prep['label'] = df_prep['key'] + '/' + df_prep['pre_obj']
df = df_prep.loc[:,['key','pred','pre_obj','label','key_vec','obj_vec','pred_vec']]

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
pd.to_pickle(df,'../../pickles/pre_exp_df.pkl')